# データセット構成モニタリング（試作版）

データセット構成モニタリングの試作版。

## 処理の流れ

- DMPの情報として、`dirStructure`が定義されている想定。何種類かのスキームから選んでもらうようなイメージでいる。
    - 現状作成しているのは、「RCOS_with_code」「RCOS_only_data」「YODA」の３種類。
- monitoring_params.jsonから`dirStructure`の値を取得。暫定的に、実験名とスキーム名の辞書型としている。
- 実験名のリストとスキーム名のリストとをそれぞれ作成し、モニタリング処理を回す。
    - 現状、スキームごとに望ましいフォルダ構成を定義しており、それぞれのパスが実際に存在するかを確認し、その結果を出力するような実装になっている。

## 議論したい点

- そもそも、DMP作成時には実験リポジトリは作成されていない筈なので、実験名をキーにとることが難しい？
- 一方で、実験ごとにディレクトリ構成が変わる場合もあるのではないか？
- 

In [ ]:
import os
import json
from typing import List


def get_ideal_paths(exp_name, scheme_name: str) -> List[str]:
    # FIXME: hard cording
    # データセットのリストを作りたい
    path_to_dataset = '/home/jovyan/experiments/' + exp_name

    # 存在することが望ましいディレクトリ・ファイルのリスト
    ideal_paths_RCOS_with_code = [
        path_to_dataset + '/source',
        path_to_dataset + '/input_data',
        path_to_dataset + '/output_data',
        path_to_dataset + '/Dockerfile',
        path_to_dataset + '/.gitattributes',
        path_to_dataset + '/.gitignore',
        path_to_dataset + '/meta_data.json',
        path_to_dataset + '/experiment.ipynb'
    ]

    ideal_paths_RCOS_only_data = [
        path_to_dataset + '/input_data',
        path_to_dataset + '/output_data',
        path_to_dataset + '/Dockerfile',
        path_to_dataset + '/.gitattributes',
        path_to_dataset + '/.gitignore',
        path_to_dataset + '/meta_data.json',
        path_to_dataset + '/experiment.ipynb'
    ]

    ideal_paths_YODA = [
        path_to_dataset + '/ci',
        path_to_dataset + '/code',
        path_to_dataset + '/code/tests',
        path_to_dataset + '/docs',
        path_to_dataset + '/docs/build',
        path_to_dataset + '/docs/source',
        path_to_dataset + '/envs',
        path_to_dataset + '/inputs',
        path_to_dataset + '/inputs/data',
        path_to_dataset + '/outputs',
        path_to_dataset + '/CHANGELOG.md',
        path_to_dataset + '/HOWTO.md',
        path_to_dataset + '/README.md',
    ]

    return eval('ideal_paths_' + scheme_name)

###########################################################################################################


def monitor_dir_structure(dataset_name: str, scheme_name: str) -> None:
    ideal_paths: List[str] = get_ideal_paths(dataset_name, scheme_name)

    print('==================================== Monitoring Dataset Structure ====================================')
    print('Dataset name: ' + dataset_name)
    print(' STATUS    PATH')

    for path in ideal_paths:
        if os.path.exists(path):
            print('  OK    ' + path)
        else:
            print('  NG    ' + path)
            print('        [WARN] Your dataset structure is deprecated.')

    print('\nMonitered by "' + scheme_name + '" scheme.\n')

###########################################################################################################

# スクリプトファイルの拡張子リストを定義（あるいは読み出し）
# something cool...


def main() -> None:
    path_params = "/home/jovyan/WORKFLOW/FLOW/param_files/monitoring_params.json"
    scheme_dict = {}
    with open(path_params, "r") as f:
        dmp = json.load(f)
        scheme_dict = dmp["dirStructure"]

    # データセットのリストと構成テンプレートのリストとを作る
    dataset_list: List[str] = [k for k, v in scheme_dict.items()]
    scheme_list: List[str] = [v for k, v in scheme_dict.items()]

    for i in range(len(dataset_list)):
        monitor_dir_structure(dataset_list[i], scheme_list[i])


if __name__ == '__main__':
    main()
